# MNIST

## 0. Import Necessary Packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(777)  # set seed for reproducibility

cuda0 = torch.device('cuda:0') # load GPU device

## 1. Import Data

In [2]:
# prevent colab bug 
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

import torchvision

# # setup for stable downloading MNIST dataset
# from six.moves import urllib
# opener = urllib.request.build_opener()
# opener.addheaders = [('User-agent', 'Mozilla/5.0')]
# urllib.request.install_opener(opener)

# # load MNIST dataset
# mnist_train = torchvision.datasets.MNIST(root='./data', download=True, train=True)

# download MNIST data
!rm -rf data
!git clone https://github.com/knamdar/data

# load MNIST dataset
mnist_train = torchvision.datasets.MNIST(root='./data', download=False, train=True)

Cloning into 'data'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 2), reused 16 (delta 2), pack-reused 0
Receiving objects: 100% (16/16), 22.07 MiB | 41.39 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [4]:
# explore the data
image, label = mnist_train[13]
print('This is a digit {}:'.format(label))
image

This is a digit 6:


In [5]:
type(image)

PIL.Image.Image

### 1) Convert images to tensors
We want to convert this PIL.Image format to torch.Tensor using the `.ToTensor()` transform. Let's do it as follows for the entire dataset:

In [6]:
mnist_train = torchvision.datasets.MNIST(root='./data', download=False, train=True, transform=torchvision.transforms.ToTensor())  

In [8]:
image, label = mnist_train[13]
print(type(image))
print(image.shape) # torch.Size([1, 28, 28]) 1: channel (only brightness in this case), 28: height, 28: width

<class 'torch.Tensor'>
torch.Size([1, 28, 28])


In [9]:
print('This image has max={}'.format(image.max()), 'and min={}'.format(image.min()))

This image has max=1.0 and min=0.0


### 2) Normalize the network inputs to be centered around 0
We want to have our network inputs centered around 0. Right now there are between 0 and 1. We can use the `torchvision.transforms.Normalize` transform to do this.

In [10]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,)),  # [0, 1] range => [-1, 1] range
])

mnist_train = torchvision.datasets.MNIST(root='./data', download=False, train=True,
                                         transform=transform)
mnist_val = torchvision.datasets.MNIST(root='./data', download=False, train=False,
                                         transform=transform)

In [11]:
image, label = mnist_train[13]
print(type(image))
print(image.shape) # torch.Size([1, 28, 28]) 1: channel (only brightness in this case), 28: height, 28: width

<class 'torch.Tensor'>
torch.Size([1, 28, 28])


In [12]:
print('This image has max={}'.format(image.max()), 'and min={}'.format(image.min()))

This image has max=1.0 and min=-1.0


In [13]:
print('training set size:\t{}'.format(len(mnist_train)))
print('validation set size:\t{}'.format(len(mnist_val)))

training set size:	60000
validation set size:	10000


### 3) Load data as batches
We want to automatically load the data as batches which we can do using the following function: `torch.utils.data.DataLoader`

In [14]:
batch_size = 512

train_loader = torch.utils.data.DataLoader(mnist_train, 
                                           batch_size=batch_size,
                                           shuffle=True,                   # shuffle training set
                                           num_workers=4,                  # turns on multi-processing loading so training is not blocked by data loading
                                           pin_memory=True)                # pin_memory allows faster transfer from CPU to GPU
val_loader = torch.utils.data.DataLoader(mnist_val, 
                                         batch_size=batch_size, 
                                         num_workers=4, 
                                         pin_memory=True)

In [15]:
# Each element yielded by `train_loader` (a Python iterable) is still a 2-tuple, 
# but now consisting of a batched image tensor, and a batched label tensor.

images, labels = next(iter(train_loader))
print('batched image tensor shape: {}'.format(images.shape)) # torch.Size([512, 1, 28, 28]) 512: batch size, 1: channel, 28: height, 28: width
print('batched label tensor shape: {}'.format(labels.shape)) # torch.Size([512]) 512: batch size

batched image tensor shape: torch.Size([512, 1, 28, 28])
batched label tensor shape: torch.Size([512])


## 2. Create a Network

Let's do so based on LeNet as in the tutorial.

In [16]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.reshape(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [18]:
net = MyNet()#.to(cuda0) # uncomment the .to(cuda0) if you want to run on GPU in Colab

In [20]:
random_input = torch.randn(123, 1, 28, 28)#, device=cuda0)
output = net(random_input)
print(output.shape) # torch.Size([123, 10]) 123: batch size, 10: number of classes

torch.Size([123, 10])


## 3. Define training and evaluating functions

In [21]:
def train(net, optim):
    net.train()
    for image_cpu, label_cpu in train_loader:
        # put data onto GPU
        image = image_cpu#.to(cuda0) # uncomment the .to(cuda0) if you want to run on GPU in Colab
        label = label_cpu#.to(cuda0) # uncomment the .to(cuda0) if you want to run on GPU in Colab
        
        # clear gradient
        optim.zero_grad()
        
        # forward through the network
        output = net(image)
        
        # compute loss and gradient
        loss = F.cross_entropy(output, label)
        loss.backward()
        
        # update parameters
        optim.step()

In [22]:
def evaluate(net):
    total = 0
    correct = 0
    
    net.eval()  # puts the network in eval mode. this is important when the 
                # network has layers that behaves differently in training and 
                # evaluation time, e.g., dropout and batch norm.
    for image_cpu, label_cpu in val_loader:
        # put data onto GPU
        image = image_cpu#.to(cuda0) # uncomment the .to(cuda0) if you want to run on GPU in Colab
        label = label_cpu#.to(cuda0) # uncomment the .to(cuda0) if you want to run on GPU in Colab
        
        with torch.no_grad():  # gradients are not tracked in this context manager
                               # since we are evaluating, gradients are not needed 
                               # and we can save some time and GPU memory.
              
            # forward through the network, and get the predicted class
            prediction = net(image).argmax(dim=-1)  
            
            total += image.size(0)  # batch size
            correct += (prediction == label).sum().item()  # `.item()` retreives a python number from a 1-element tensor
            
    return correct / total

In [23]:
# Without any training, the network accuracy matches that of random guessing: ~10%.

print('At initialization, the network has accuracy {:.4f}%'.format(evaluate(net) * 100))

At initialization, the network has accuracy 7.8200%


## 4. Training and Evaluating the Model

In [24]:
num_epochs = 10
lr = 0.1

optim = torch.optim.SGD(net.parameters(), lr=lr)

for epoch in range(num_epochs):
    print('Epoch: {}\tValidation Accuracy: {:.4f}%'.format(epoch, evaluate(net) * 100))
    train(net, optim)

print('Done! \tValidation Accuracy: {:.4f}%'.format(evaluate(net) * 100))

Epoch: 0	Validation Accuracy: 7.8200%
Epoch: 1	Validation Accuracy: 75.0400%
Epoch: 2	Validation Accuracy: 91.4300%
Epoch: 3	Validation Accuracy: 96.0000%
Epoch: 4	Validation Accuracy: 96.1800%
Epoch: 5	Validation Accuracy: 94.7800%
Epoch: 6	Validation Accuracy: 95.0600%
Epoch: 7	Validation Accuracy: 96.6200%
Epoch: 8	Validation Accuracy: 94.7000%
Epoch: 9	Validation Accuracy: 98.4000%
Done! 	Validation Accuracy: 97.5400%


In [ ]:
import PIL

# Utility function
def tensor2image(tensor):
    return PIL.Image.fromarray((tensor.detach() * 127.5 + 128).clamp_(0, 255).to('cpu', torch.uint8).numpy()[0]) # 

In [ ]:
# Get a data sample we want to attack

data, label = mnist_val[30]
data = data.to(cuda0)

In [ ]:
# Look at the image

tensor2image(data)

In [ ]:
# Label is...

print('label is', label)

In [ ]:
# Let's see what our trained network says about this image.


# turn the network into eval mode
net.eval()
output = net(data.unsqueeze(0))  # .unsqueeze(0) insert a batch dimension so it looks like batched data
prediction = output.argmax(-1)
print('predicted class is', prediction.item())